In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# own Modules 
from models_sub_net_ls import LstmMse_LatentSpace, AnalysisLayer
from data_preperator import DataPreperator
from trainer import TrainerLatentSpaceAnalyser
from loss_module import LossMse
from data_set import DataSet

## Take care of these things before training:
- Select correct path and define droped_features
- Change parameter of model
- Change step_size in cycling_lr
- Change filed_location

## Parameters phm Data

In [2]:
param = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path" : '../../../data/phm_data_challenge/01_M01_DC_training.csv' ,
    },
    "preprocessing" : {
        "first_order_difference" : False,
        "droped_features": ["ID", "stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 13,
        "n_hidden_lstm" : 100,
        "sequence_size" : 150,
        "batch_size" : 8,
        "lstm_layer" : 2,
        "n_hidden_fc_pred": 50,
        "n_hidden_fc_ls": 7,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2
    },
    "cycling_lr" : {
        "scheduler_active" : True, 
        # step_size is the number of training iterations (total samples/batch_size) per half cycle. 
        # Authors suggest setting step_size 2-8 x training iterations in epoch.
        "step_size" : (12500/8)*2, 
        # Mode can be one of {triangular, triangular2, exp_range}
        "mode" : "triangular", 
        "gamma" : 0.9995,
        "base_lr" : 0.016, 
        "max_lr" :0.07
    },
    "training": {
        "n_epochs" : 100,
        "patience" : 10,
    },
    "filed_location": {
        "trained_model" : "../../../models/MSE_latent_space/phm_data",
        "history" : "../../visualisation/files/history_training/multi_learning.csv"
    }
}

## Parameters artifical signals

In [2]:
param = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path" : '../../../data/artifical_signals/MLE_analysis/artifical_2_signals.csv' ,
    },
    "preprocessing" : {
        "first_order_difference" : False,
        "droped_features": ["ID"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 2,
        "n_hidden_lstm" : 15,
        "sequence_size" : 25,
        "batch_size" : 8,
        "lstm_layer" : 1,
        "n_hidden_fc_pred": 75,
        "n_hidden_fc_ls": 7,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2
    },
    "cycling_lr" : {
        "scheduler_active" : True, 
        # step_size is the number of training iterations (total samples/batch_size) per half cycle. 
        # Authors suggest setting step_size 2-8 x training iterations in epoch.
        "step_size" : (12500/8)*2, 
        # Mode can be one of {triangular, triangular2, exp_range}
        "mode" : "triangular", 
        "gamma" : 0.9995,
        "base_lr" : 0.016, 
        "max_lr" :0.07
    },
    "training": {
        "n_epochs" : 100,
        "patience" : 10,
    },
    "filed_location": {
        "trained_model" : "../../../models/MSE_latent_space/artifical_2_signals",
        "history" : "../../visualisation/files/history_training/multi_learning.csv"
    }
}

## Preprocessing
1. First order difference (if selected)
2. Split data into train and validation data
3. Scale train and validation data with train's mean and variance

In [3]:
train_loader = DataPreperator(path=param['data']['path'], 
                              ignored_features=param['preprocessing']['droped_features'],
                              stake_training_data=param['data']['stake_training_data'],
                              features_not_to_scale=param['preprocessing']['features_not_to_scale'],
                              first_order_difference=param["preprocessing"]["first_order_difference"])
train_data, validation_data = train_loader.prepare_data()
print(len(train_data)+len(validation_data))

12000


### Mean and variance from scale process (only of continious features)

In [4]:
mean, var = train_loader.provide_statistics()
print(mean)
print(var)

[-0.00393712 -0.01294209]
[49.18936568  0.34270256]


## Initialize Dataset 
Time series data must be transformed into a structure of samples with `input` and `target` components before it can be used to fit a supervised learning model. <br>
For a time series interested in one-step predictions, the observations at prior time steps, so-called lag observations, are used as `input` and the `target` is the observation at the current time step.

For example, a univariate series can be expressed as a supervised learning problem with three time steps for `input` and one step as `target`, as follows:

|input|target|
|-----|------|
[1, 2, 3]|[4]
[2, 3, 4]|[5]
[3, 4, 5]|[6]

The Keras deep learning library provides the `TimeseriesGenerator` to automatically transform both univariate and multivariate time series data into such a format. 

In [5]:
dataset_train = DataSet(train_data, timesteps=param["model"]["sequence_size"])
dataset_validation = DataSet(validation_data, timesteps=param["model"]["sequence_size"])

## Initialize DataLoader
Actually the data has a other size than in the table above because of multivariate time series data and because of using batches. <br>
__First dimension__: batch size --> Defines the number of samples that will be propagated through the network simultaneously. <br>
__Second dimension:__ timesteps --> Number of sequence which is passed into the LSTM <br>
__Third dimension:__ input_dim --> Is the number of features. In this case data from 7 sensors, collected at the same time. <br>

![](../../../knowledge/pictures/input_shape.png)

Data is shuffled because each mini batch is indipendent from each other, but samples of a minibatch are in chronological order.

In [6]:
data_loader_training = DataLoader(dataset_train, 
                                  batch_size=param["model"]["batch_size"], 
                                  num_workers=0, 
                                  shuffle=True, 
                                  drop_last=True
                                 )
data_loader_validation = DataLoader(dataset_validation, 
                                    batch_size=param["model"]["batch_size"], 
                                    num_workers=0, 
                                    shuffle=True, 
                                    drop_last=True
                                   )

In [7]:
for batch_idx, data in enumerate(data_loader_training):
    x,y = data
    print('Data of batch: {}'.format(batch_idx))
    print("Size of input data: {}".format(x.size()))
    print("Size of target data: {}".format(y.size()))
    if batch_idx >=1: break

Data of batch: 0
Size of input data: torch.Size([8, 25, 2])
Size of target data: torch.Size([8, 2])
Data of batch: 1
Size of input data: torch.Size([8, 25, 2])
Size of target data: torch.Size([8, 2])


## Initialize Neural Network
__Parameters for LSTM Modul:__
- input_size : The number of expected features in the input x
- hidden_size :The number of features in the hidden state h
- num_layers : Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results.
- batch_first : If True, then the input __and output__ tensors are provided as (batch, seq, feature).
- dropout – If non-zero, introduces a Dropout layer on the outputs of each LSTM layer except the last layer, with dropout probability equal to dropout. Default: 0

In [9]:
torch.manual_seed(0)
model = LstmMse_LatentSpace(batch_size=param['model']['batch_size'], 
                           input_dim=param['model']['input_size'], 
                           n_hidden_lstm=param['model']['n_hidden_lstm'], 
                           n_layers=param['model']['lstm_layer'],
                           dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                           dropout_rate_fc= param['model']['dropout_rate_fc'],
                           n_hidden_fc_prediction=param['model']['n_hidden_fc_pred'], 
                           n_hidden_fc_ls_analysis=param['model']['n_hidden_fc_ls']      
                           )

## Define same MSE Loss function for both sub networks

In [14]:
criterion_prediction = LossMse(param["model"]["input_size"], param["model"]["batch_size"])
criterion_ls = LossMse(param["model"]["input_size"], param["model"]["batch_size"])

## Initialize Optimizer and Cyclic Learning Rate Scheduler

In [15]:
optimizer = torch.optim.SGD(model.parameters(), lr=1.)  
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer, 
                                              base_lr=param['cycling_lr']['base_lr'], 
                                              max_lr=param['cycling_lr']['max_lr'], 
                                              step_size_up=param['cycling_lr']['step_size'], 
                                              mode=param['cycling_lr']['mode'],
                                              gamma=param['cycling_lr']['gamma']
                                             )

## Initialize Trainer

In [16]:
trainer = TrainerLatentSpaceAnalyser(model=model,
                                     optimizer=optimizer,
                                     scheduler=scheduler,
                                     scheduler_active = param["cycling_lr"]["scheduler_active"],
                                     criterion_prediction = criterion_prediction,
                                     criterion_ls = criterion_ls,
                                     location_model=param["filed_location"]["trained_model"], 
                                     location_stats=param["filed_location"]["history"], 
                                     patience=param['training']['patience']
                                     )

## Training Loop
An epoch consists of a learning cycle over all batches of training data and an evaluation of the most recent model with the testing data. 

In [17]:
# Create lists to save training loss and validation loss of each epoch
hist_loss = []
torch.manual_seed(0)
print("Start training.")

for epoch in range(param['training']['n_epochs']):
    # Train with batches 
    mean_epoch_training_loss = trainer.train(data_loader_training)

    # Evaluate
    mean_epoch_validation_loss = trainer.evaluate(data_loader_validation, hist_loss, epoch)

    # Cache History
    trainer.cache_history_training(hist_loss, epoch, mean_epoch_training_loss, mean_epoch_validation_loss)

    # Save model if its the best one since the last change in configuration of hyperparameters
    status_ok = trainer.save_model(epoch, mean_epoch_validation_loss, param['model']['input_size'], 
                                   param['model']['lstm_layer'], param['model']['n_hidden_lstm'], 
                                   param['model']['n_hidden_fc_pred'], param["model"]["sequence_size"],
                                   param['model']['n_hidden_fc_ls']
                                  )
    if not status_ok:
        break

# Safe results to csv file
df = pd.DataFrame(hist_loss)
df.to_csv(param["filed_location"]["history"], sep=";", index=False)

Start training.


KeyboardInterrupt: 